In [ ]:
import random
import time
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pds
import datetime as dt

In [ ]:
DOFILE="run.do.in"
RAM1={
"instance":"testbench/lpp_lfr_filter_1/IIR_CEL_CTRLR_v2_1/IIR_CEL_CTRLR_v2_DATAFLOW_1/RAM_CTRLR_v2_1/memRAM/SRAM/axc/x0/a8to12/agen(0)/u0/u0/MEMORYFILE",
"abits":8,
"dbits":12,
"name":"RAM1.txt"
}
RAM2={
"instance":"testbench/lpp_lfr_filter_1/IIR_CEL_f0_to_f1/IIR_CEL_CTRLR_v2_DATAFLOW_1/RAM_CTRLR_v2_1/memRAM/SRAM/axc/x0/a8to12/agen(0)/u0/u0/MEMORYFILE",
"abits":8,
"dbits":12,
"name":"RAM2.txt"
}
RAM3={
"instance":"testbench/lpp_lfr_filter_1/cic_lfr_1/memRAM/SRAM/axc/x0/a8to12/agen(0)/u0/u0/MEMORYFILE",
"abits":9,
"dbits":10,
"name":"RAM3.txt"
}
RAM4={
"instance":"testbench/lpp_lfr_filter_1/cic_lfr_1/memRAM/SRAM/axc/x0/a8to12/agen(1)/u0/u0/MEMORYFILE",
"abits":9,
"dbits":10,
"name":"RAM4.txt"
}
RAM5={
"instance":"testbench/lpp_lfr_filter_1/YES_IIR_FILTER_f2_f3/IIR_CEL_CTRLR_v3_1/RAM_CTRLR_v2_1/memRAM/SRAM/axc/x0/a8to12/agen(0)/u0/u0/MEMORYFILE",
"abits":8,
"dbits":12,
"name":"RAM5.txt"
}
RAM6={
"instance":"testbench/lpp_lfr_filter_1/YES_IIR_FILTER_f2_f3/IIR_CEL_CTRLR_v3_1/RAM_CTRLR_v2_2/memRAM/SRAM/axc/x0/a8to12/agen(0)/u0/u0/MEMORYFILE",
"abits":8,
"dbits":12,
"name":"RAM6.txt"
}

RAMS=[RAM1,RAM2,RAM3,RAM4,RAM5,RAM6]

In [ ]:
def mkram(length,width,gentype='rand',**kwargs):
    return toBinStr(gen(length,width,gentype,**kwargs),width)

def toBinStr(data,width):
    return [format(val, 'b').zfill(width) for val in data]

def gen(length,width,gentype='rand',**kwargs):
    LUT={
        "rand":gen_rand,
        "const":gen_const
    }
    return LUT[gentype](length,width,**kwargs)

def gen_rand(length,width,**kwargs):
    random.seed(time.time())
    mask=(2**width)-1
    data=[]
    for line in range(length):
        data.append(int(2**32*random.random())&mask)
    return data

def gen_const(length,width, value):
    mask=(2**width)-1
    return [value&mask for i in range(length)]

def save(data,file):
    f = open(file,"w")
    [f.write(line+'\n') for line in data]
    f.close()
    
def start_Vsim(gentype='rand',**kwargs):
    args=""
    for RAM in RAMS:
        save(mkram(2**RAM["abits"],RAM["dbits"],gentype=gentype,**kwargs),"simulation/"+RAM["name"])
        args = args + "  -g" + RAM["instance"] + "=\"" + RAM["name"] + "\""
    with open("run.do.in","r") as inFile, open("simulation/run.do","w") as outFile:
        input = inFile.read()
        outFile.write(input.replace("#VSIM_ARGS#",args))
    $(cd simulation)
    vsim -do run.do > sim.log
    folder=dt.datetime.today().strftime("%Y-%m-%d_%H-%M")
    mkdir @(folder)
    for RAM in RAMS:
        cp @(RAM["name"]) @(folder+"/"+RAM["name"])
    cp log*.txt run.do sim.log @(folder)   
    $(cd ..)
    


In [ ]:
df = pds.read_csv("./simulation/log_input.txt",index_col=0,delim_whitespace=True,header=None,names=["TSTAMP","BIAS1","BIAS2","BIAS3","BIAS4","BIAS5","B1","B2","B3"])
df2 = pds.read_csv("./simulation/log_output_f0.txt",index_col=0,delim_whitespace=True,header=None,names=["TSTAMP","BIAS1","BIAS4","BIAS5","B1","B2","B3"])
df3 = pds.read_csv("./simulation/log_output_f1.txt",index_col=0,delim_whitespace=True,header=None,names=["TSTAMP","BIAS1","BIAS4","BIAS5","B1","B2","B3"])
df4 = pds.read_csv("./simulation/log_output_f2.txt",index_col=0,delim_whitespace=True,header=None,names=["TSTAMP","BIAS1","BIAS4","BIAS5","B1","B2","B3"])

df.index*=5e-9
if len(df2.index):
    df2.index*=5e-9
    df2/=0.89
if len(df3.index):
    df3.index*=5e-9
    df3/=0.87
if len(df4.index):
    df4.index*=5e-9
    df4/=0.89

print(len(df))
df.filter(["B1"]).plot()
#plt.plot(df2)
plt.plot(df3.filter(["B1"]))
#plt.plot(df4)
plt.show()

In [ ]:
cd ..

In [ ]:
mkram(2,32)

mkram(20,32,gentype='const',value=55)

save(mkram(10,32),"RAM_FILE.txt")

In [ ]:
for i in range(2):
    start_Vsim(gentype='rand',value=0)